# Práctica 3 parte 2
##### Isabela Trujillo - Laura Vargas
Se cuenta con señales de EEG de dos grupos de personas, un  grupo control y un grupo de pacientes con enfermedad de Parkinson. Se tiene evidencia que la energía de las señales de EEG puede conducir a diferenciar entre señales de pacientes con enfermedad de Parkinson y sanos. Se pide calcular la Energía promedio por grupo poblacional de cada canal.

In [7]:
import glob
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import scipy.stats as sis
import scipy.io as sio
import statsmodels.tsa.stattools as sta
from statsmodels.tsa.stattools import adfuller
from scipy.stats import mannwhitneyu
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import ttest_ind
import os
import zipfile
import io
import seaborn as sns


##### Implemente una función que reciba una señal de múltiples canales y épocas y calcule la Energía de promedio de cada canal

In [22]:

ruta=os.getcwd() 

with zipfile.ZipFile(ruta+'\\datos_senales_datos_parkinson_cursos.zip', 'r') as zip_ref:
  for file in zip_ref.infolist():
    if file.filename.startswith('datos_senales_datos_parkinson_cursos/control/C0') and file.filename.endswith('.mat'):
          with zip_ref.open(file) as ts_file:
                    ts_file_contents = sio.loadmat(ts_file)
                    ts_file_contents=ts_file_contents['data']
                    senal_continua_c = np.reshape(ts_file_contents,(ts_file_contents.shape[0],ts_file_contents.shape[1]*ts_file_contents.shape[2]),order='F')
                    senal_continua_c= senal_continua_c.T

    #senal_continua_c=pd.DataFrame(senal_continua_c)
    if file.filename.startswith('datos_senales_datos_parkinson_cursos/parkinson/P0') and file.filename.endswith('.mat'):
          with zip_ref.open(file) as ts_file:
                    ts_file_contents = sio.loadmat(ts_file)
                    ts_file_contents=ts_file_contents['data']
                    senal_continua_p = np.reshape(ts_file_contents,(ts_file_contents.shape[0],ts_file_contents.shape[1]*ts_file_contents.shape[2]),order='F')
                    senal_continua_p = senal_continua_p.T
          #senal_continua_p=pd.DataFrame(senal_continua_p)           


print(f'park: \n{senal_continua_p}\n control: \n{senal_continua_c}')


park: 
[[ 0.7801739   0.35548592 -1.23701763 ...  8.61526585  5.32485151
   2.10171628]
 [ 0.7717725   0.32634103 -1.17328632 ...  8.52684021  5.27745581
   2.00372195]
 [ 0.75064629  0.28169858 -1.11551464 ...  8.40527725  5.22155333
   1.92058063]
 ...
 [-2.35174608 -0.37010911  2.63554406 ... -3.26450038 -1.90368557
  -2.75989079]
 [-2.56928277 -0.46061552  2.74045372 ... -3.29805875 -1.99913025
  -2.91318083]
 [-2.77679467 -0.54325163  2.83435702 ... -3.342592   -2.09891915
  -3.07217503]]
 control: 
[[-1.08502638 -2.61430597 -1.10882771 ... -0.63866556  1.22497225
   2.94287109]
 [-0.87366116 -2.42915678 -0.90971917 ... -0.41136748  1.32911432
   2.92242956]
 [-0.66055715 -2.25657654 -0.73941201 ... -0.13530292  1.46772242
   2.93172073]
 ...
 [ 6.33238554  1.24629378 -1.81045485 ... -5.15402412 -8.70972729
  -8.77152061]
 [ 6.76563406  1.56646693 -1.66290319 ... -5.53995752 -9.0186224
  -9.2203083 ]
 [ 7.16178656  1.86495924 -1.51589954 ... -5.86591959 -9.29266548
  -9.64237785]]

In [24]:
def energia(senal):
    promCanales = []
    for canal in senal:
        energia = np.sum(np.square(canal))
        energia_m = np.mean(energia)
        promCanales.append(energia_m)
    return promCanales

e_control=energia(ts_file_contents)
e_parkinson=energia(senal_continua_p)
print(f'La energía de control:\n {pd.DataFrame(e_control)}\n La energía de Parkinson: \n{pd.DataFrame(e_parkinson)}')

La energía de control:
               0
0  2.720273e+06
1  2.325277e+06
2  2.724161e+06
3  2.461960e+06
4  2.503053e+06
5  5.345319e+06
6  4.792634e+06
7  5.629962e+06
 La energía de Parkinson: 
                 0
0       135.328438
1       133.200384
2       130.348094
3       126.744488
4       122.386026
...            ...
339995   31.399199
339996   34.322731
339997   37.554865
339998   41.037077
339999   44.697274

[340000 rows x 1 columns]


##### Calcule la energía de cada canal promediada por épocas para cada sujeto, esto para ambos grupos poblacionales. Guarde esta información en un DataFrame de columnas ‘canal’ y filas ‘#sujeto’ con los valores de energía calculados, un DataFrame para cada grupo poblacional. 

In [25]:
def e_canal(datos):
    titulos=['1','2','3','4','5','6','7','8']
    
    canales=[]
    for i in range(len(datos)):
        k=datos[i]
        canales.append(energia(k))
    t_c=pd.DataFrame(columns=titulos)
    for x, sujeto in enumerate(canales,1):
        t_c.loc[x]=sujeto
    return t_c

e_canal_control=e_canal(senal_continua_c)
e_canal_parkinson=e_canal(senal_continua_p)
#print(f'La energía de control:\n {e_canal_control}\n La energía de Parkinson:\n {e_canal_parkinson}')

KeyboardInterrupt: 

##### Determine si existe diferencia estadística entre canales de cada grupo de sujetos a través de una prueba t. Compruebe los supuestos necesarios para realizar una prueba t, esto es: Normalidad de la variable, independencia (se asume que los grupos son independientes), y homocedasticidad (use una prueba de Levene), finalmente realice la prueba t para determinar si existen diferencias entre los canales entre grupos de sujetos.

In [ ]:
resultados = []

for canal in e_canal_control.columns:
    print(f"Análisis para el canal: {canal}")
    c_control = e_canal_control[canal]
    c_parkinson = e_canal_parkinson[canal]

    
    shapiro_control = sis.shapiro(c_control)
    shapiro_parkinson = sis.shapiro(c_parkinson)

    normal = shapiro_control.pvalue > 0.05 and shapiro_parkinson.pvalue > 0.05
    print(f"  Prueba de normalidad\n    p-value Control: {shapiro_control.pvalue}\n    p-value Parkinson: {shapiro_parkinson.pvalue}")

    prueba_levene = sis.levene(c_control, c_parkinson)
    homogeneo = prueba_levene.pvalue > 0.05
    print(f"  Prueba de Levene (homocedasticidad)\n    p-value: {prueba_levene.pvalue}")


Si cumple con normalidad y homocedasticidad se realiza la prueba t

In [ ]:

for canal in e_canal_control.columns:
    prueba_t = sis.ttest_ind(c_control, c_parkinson)
    print(f"  Prueba t - p-value: {prueba_t.pvalue}")
    resultados.append((canal, "t-test", prueba_t.pvalue))

Si No cumple con normalidad y homocedasticidad se realiza la prueba de Mann Whitney

#####  De no cumplirse los requisitos, realice entonces un análisis no paramétrico (prueba U de Mann-Whitney). Este numeral tiene como objetivo identificar los canales que entregan información diferencial entre pacientes Sanos y con enfermedad de Parkinson.

In [ ]:
for canal in e_canal_control.columns:
    prueba_mann = sis.mannwhitneyu(c_control, c_parkinson)
    print(f"  Prueba U de Mann-Whitney - p-value: {prueba_mann.pvalue}")
    resultados.append((canal, "Mann-Whitney", prueba_mann.pvalue))